<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/2_Transformaciones_basicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proceso completo de Extracción, Transformación y Carga
Partimos del fichero con datos en bruto llamado **empleados_metacortex.csv**

In [57]:
!git clone https://github.com/franciscogarate/cdiae

In [58]:
import pandas as pd
file = 'cdiae/data/01_raw/empleados_metacortex.csv'
df = pd.read_csv(
        file,
        sep=';',                    # Separador de campos
        header=3,                   # Fila 4 contiene los encabezados (0-indexed, salta comentarios)
        encoding='utf-8',           # Codificación de caracteres
        skipfooter=1,               # Omite la última 1 fila (comentarios finales)
        engine='python',            # Necesario para skipfooter
        comment='%',                # Líneas que empiecen con % son comentarios
        usecols=['fecha_nacimiento','fecha_alta','nombre','sexo_biologico','nif','codigo_postal','genero','departamento','activo','horas_semanales','bonus'],
        parse_dates=['fecha_nacimiento', 'fecha_alta'],  # Convierte automáticamente a datetime
        date_format='%Y-%m-%d',      # OJO: Formato único para todas las columnas de fecha
        na_values=['', 'N/A', 'null', 'NULL'],
        keep_default_na=True,
)
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], format='%Y/%m/%d')
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,Mujer,Actuarial,Sí,40,5000.0
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,Hombre,IT,Sí,35h,NaN
2,1979-09-19,2004-03-01,Fran Garcia,M,29015,NaN,M,Finanzas,Sí,40,NaN
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,NaN,Finanzas,No,40h,3000.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,Mujer,Actuarial,Sí,45,NaN
5,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,NaN,IT,Sí,45,6000.0
6,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,Mujer,IT,Sí,38,5000.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,Mujer,IT,Sí,35,7000.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,Hombre,Finanzas,Sí,35,NaN


### Cruce de datos con otros ficheros
Partiendo de nuestros datos, vamos a complementarlos con información externa como los pagos efectuados que residen en un fichero de Excel externo.
Para ello utilizaremos las siguientes funciones de pandas:
  - `DataFrame.groupby`
  - `DataFrame.reset_index`
  - `pd.merge`

Lectura del Excel con pagos de empleados:

In [59]:
df_pagos = pd.read_excel('cdiae/data/01_raw/pagos_empleados_metacortex.xlsx')
df_pagos.head()

,FECHA,NIF,IMPORTE
0,2025-03-19,88862218Q,240
1,2025-03-20,17905733A,108
2,2025-03-21,52745245N,69
3,2025-03-22,82167366G,196
4,2025-03-23,28859641T,215


Renombramos columnas para coincidir con el nombre de las columnas de 'df' antes del cruce

In [60]:
df_pagos.rename(columns={'NIF': 'nif', 'IMPORTE': 'pago'}, inplace=True)
df_pagos.head()

,FECHA,nif,pago
0,2025-03-19,88862218Q,240
1,2025-03-20,17905733A,108
2,2025-03-21,52745245N,69
3,2025-03-22,82167366G,196
4,2025-03-23,28859641T,215


Inspeccionamos los datos por su tipo, aquellos nulos y la memoria del DataFrame de pagos

In [61]:
df_pagos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   25 non-null     datetime64[ns]
 1   nif     25 non-null     object        
 2   pago    25 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 732.0+ bytes


Agregamos pagos por empleado (nif) sumando todos sus registros

In [62]:
df_pagos_agg = df_pagos.groupby('nif')['pago'].sum().reset_index()
df_pagos_agg

,nif,pago
0,17905733A,611
1,28859641T,938
2,52745245N,1074
3,82167366G,1010
4,88862218Q,733


Hacemos un left join para añadir 'pago' al DataFrame principal

In [63]:
df = pd.merge(df, df_pagos_agg, on=['nif'], how='left')
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,Mujer,Actuarial,Sí,40,5000.0,733.0
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,Hombre,IT,Sí,35h,NaN,611.0
2,1979-09-19,2004-03-01,Fran Garcia,M,29015,NaN,M,Finanzas,Sí,40,NaN,NaN
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,NaN,Finanzas,No,40h,3000.0,1074.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,Mujer,Actuarial,Sí,45,NaN,1010.0
5,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,NaN,IT,Sí,45,6000.0,NaN
6,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,Mujer,IT,Sí,38,5000.0,938.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,Mujer,IT,Sí,35,7000.0,938.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,Hombre,Finanzas,Sí,35,NaN,NaN


## Distintos procesos de transformación de datos
### Map y replace
Unas de las funciones que podemos utilizar para reemplazar y unificar los valores posibles son `map` y `replace`.
Adicionalmente, para poder todos los valores que posee un campo podemos usar las funciones `unique` y `nunique`.

In [64]:
df.sexo_biologico.unique()

array(['F', 'M'], dtype=object)

Valores únicos de la columna 'genero' antes del mapeo

In [65]:
df.genero.unique()

array(['Mujer', 'Hombre', 'M', nan], dtype=object)

Homogeneizamos etiquetas: 'Mujer'->'F' y 'Hombre'->'M'

In [66]:
df['genero'] = df['genero'].map({'Mujer':'F', 'Hombre':'M'})
df.genero.unique()                                                  # Valores únicos tras el mapeo

array(['F', 'M', nan], dtype=object)

In [67]:
df.activo.unique()                                                  # Valores únicos actuales de 'activo'

array(['Sí', 'No'], dtype=object)

### Reemplazar con un condicional

Ejemplo de conversión con apply (mantener comentado si no se necesita)

In [68]:
df['activo'] = df.apply(lambda x: True if x.activo == 'Sí' else False, axis=1)
df

,fecha_nacimiento,fecha_alta,nombre,sexo_biologico,codigo_postal,nif,genero,departamento,activo,horas_semanales,bonus,pago
0,1990-01-15,2020-01-15,Ana García,F,28013,88862218Q,F,Actuarial,True,40,5000.0,733.0
1,1985-12-03,2019-12-03,Luis Rodríguez,M,41001,17905733A,M,IT,True,35h,NaN,611.0
2,1979-09-19,2004-03-01,Fran Garcia,M,29015,NaN,NaN,Finanzas,True,40,NaN,NaN
3,1992-07-22,2021-07-22,María López,F,48004,52745245N,NaN,Finanzas,False,40h,3000.0,1074.0
4,1988-04-10,2018-04-10,Carlos Martín,M,48001,82167366G,F,Actuarial,True,45,NaN,1010.0
5,1981-03-11,2018-04-10,Luis González,M,48007,1248870G,NaN,IT,True,45,6000.0,NaN
6,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,38,5000.0,938.0
7,1995-11-08,2022-11-08,Elena Ruiz,F,15003,28859641T,F,IT,True,35,7000.0,938.0
8,1986-05-08,1922-03-04,Gorka de Miguel,M,46001,11859641K,M,Finanzas,True,35,NaN,NaN


Resumen de tipos de datos y memoria tras las transformaciones:

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fecha_nacimiento  9 non-null      datetime64[ns]
 1   fecha_alta        9 non-null      datetime64[ns]
 2   nombre            9 non-null      object        
 3   sexo_biologico    9 non-null      object        
 4   codigo_postal     9 non-null      int64         
 5   nif               8 non-null      object        
 6   genero            6 non-null      object        
 7   departamento      9 non-null      object        
 8   activo            9 non-null      bool          
 9   horas_semanales   9 non-null      object        
 10  bonus             5 non-null      float64       
 11  pago              6 non-null      float64       
dtypes: bool(1), datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 933.0+ bytes


## Guardado de datos
- El último paso del proceso de ETL es la carga o guardado (loading). Para ello, no vamos a utilizar el mismo formato del fichero origen (raw) sino que dicho fichero transformado, siguiendo los estandares de buenas prácticas, vamos a guardarlo como ficheros intermedios en un formato que incorporé el formato.
- La mejor opción para ficheros intermedios son los formatos feather y parquet.
- Veamos las principales caracteristicas, así como pros y contras de cada uno de ellos.
### Feather
El formato Feather es un formato de archivo binario para almacenar datos de forma eficiente. Es un formato abierto y ligero que se utiliza para el intercambio de datos entre aplicaciones y lenguajes de programación.
Así, es el lenguage ideal para guardar nuestras base de datos _intermedias_ en nuestro flujo de transformaciones en python, así como para compartir dichos ficheros con otros programas escritos en R, ya que R tambien lee sin problemas el formato Feather.
### Parquet
El formato Parquet, en cambio, es un formato de archivo columnar (en columnas) muy eficiente que se utiliza para almacenar datos principalmente de texto. Tambien es un formato abierto y ligero que se utiliza para el intercambio de datos entre aplicaciones y lenguajes de programación.
- Ambos tienen compatibilidad con pandas, pero Parquet es más eficiente para grandes volúmenes de datos.
    - `df.to_parquet('ruta/archivo.parquet')`
    - `df.to_feather('ruta/archivo.feather')`
- Feather y Parquet mantienen los tipos de datos originales (int64, float64, datetime, etc.), a diferencia de CSV que convierte todo a texto.
- Las principales diferencias son:
    - Feather está diseñado específicamente para ser extremadamente rápido en lectura/escritura
    - Feather usa un formato binario sencillo basado en Apache Arrow, lo que lo hace ideal para intercambio rápido entre Python y R.
    - Parquet utiliza una compresión mas compleja pensada en reducir el tamaño
    - Feather tiene menor compresión que Parquet, resultando en archivos más grandes, pero con acceso más rápido.
- Existe tambien el formato pickle aunque es especifico para python
- Mi recomendación para ficheros temporales de cualquier tamaño es Feather, y Parquet para ficheros muy pesados que vayan a almacenarse en la nube (> 10 GB) o dataset con muchas columnas, ya que en estos casos, Parquet puede llegar a comprimir hasta 10 veces.
- Cuando trabajas con fichero .parquet no puedes importar las x primeras lineas.

In [70]:
df.to_feather('cdiae/data/02_intermediate/empleados_metacortex.ftr')   # Guardamos el dataset intermedio en formato Feather